In [ ]:
open System

let private random = Random ()

let rollD6 () = random.Next (1, 7)

In [ ]:
let rollMax fn max n =
    [1..n]
    |> List.map (fun _ -> fn false max)
    |> List.groupBy id
    |> List.map (fun (_, v) -> v.Length)

### Inverse Transform Sampling

In [ ]:
let inverseTransformSamplingRoll log max =
    let rec numDice n =
        if log then printfn $"numDice / n: {n} / max: {max} / pown: {pown 6 n}"
        if pown 6 n >= max
        then n
        else numDice (n + 1)

    let rec rollAcc acc power =
        if power < 0
        then
            if log then printfn $"rollAcc / acc: {acc} / power: {power}"
            acc
        else
            let roll = rollD6 () - 1
            let value = roll * (pown 6 power)
            if log then printfn $"rollAcc / acc: {acc} / power: {power} / roll: {roll} / value: {value}"
            rollAcc (acc + value) (power - 1)

    let rec rollMax power =
        let result = rollAcc 0 power
        if log then printfn $"rollMax / power: {power} / result: {result}"
        if result < max
        then result + 1
        else rollMax power

    rollMax ((numDice 1) - 1)

In [ ]:
//// ignore

inverseTransformSamplingRoll true 2000

34

numDice / n: 1 / max: 2000 / pown: 6
numDice / n: 2 / max: 2000 / pown: 36
numDice / n: 3 / max: 2000 / pown: 216
numDice / n: 4 / max: 2000 / pown: 1296
numDice / n: 5 / max: 2000 / pown: 7776
rollAcc / acc: 0 / power: 4 / roll: 4 / value: 5184
rollAcc / acc: 5184 / power: 3 / roll: 3 / value: 648
rollAcc / acc: 5832 / power: 2 / roll: 2 / value: 72
rollAcc / acc: 5904 / power: 1 / roll: 0 / value: 0
rollAcc / acc: 5904 / power: 0 / roll: 4 / value: 4
rollAcc / acc: 5908 / power: -1
rollMax / power: 4 / result: 5908
rollAcc / acc: 0 / power: 4 / roll: 2 / value: 2592
rollAcc / acc: 2592 / power: 3 / roll: 3 / value: 648
rollAcc / acc: 3240 / power: 2 / roll: 0 / value: 0
rollAcc / acc: 3240 / power: 1 / roll: 1 / value: 6
rollAcc / acc: 3246 / power: 0 / roll: 0 / value: 0
rollAcc / acc: 3246 / power: -1
rollMax / power: 4 / result: 3246
rollAcc / acc: 0 / power: 4 / roll: 4 / value: 5184
rollAcc / acc: 5184 / power: 3 / roll: 1 / value: 216
rollAcc / acc: 5400 / power: 2 / roll: 0 / 

In [ ]:
//// ignore

[1..100]
|> List.map (fun _ -> inverseTransformSamplingRoll false 10)
|> List.groupBy id
|> List.map (fun (k, v) -> k, v.Length)
|> List.sortBy fst

index value 0 (1, 9) Item1 1 Item2 9 1 (2, 12) Item1 2 Item2 12 2 (3, 11) Item1 3 Item2 11 3 (4, 12) Item1 4 Item2 12 4 (5, 9) Item1 5 Item2 9 5 (6, 11) Item1 6 Item2 11 6 (7, 10) Item1 7 Item2 10 7 (8, 12) Item1 8 Item2 12 8 (9, 7) Item1 9 Item2 7 9 (10, 7) Item1 10 Item2 7

In [ ]:
//// ignore

let max = 10
let n = 30
let even = (n / max) |> int

let rec inverseTransformSamplingRollN current =
    let roll = rollMax inverseTransformSamplingRoll max n
    if roll |> List.forall ((=) even)
    then current
    else inverseTransformSamplingRollN (current + 1)

inverseTransformSamplingRollN 0

731444

In [ ]:
//// ignore

[1..100] |> List.map (fun _ -> inverseTransformSamplingRollN 0) |> List.map float |> List.average

265904.38

In [ ]:
let inverseTransformSamplingRoll2 log max =
    let rec numDice n =
        if log then printfn $"numDice / n: {n} / max: {max} / pown: {pown 6 n}"
        if pown 6 n > max
        then n
        else numDice (n + 1)

    let rec rollAcc acc power =
        if power < 0
        then
            if log then printfn $"rollAcc / acc: {acc} / power: {power}"
            acc
        else
            let roll = rollD6 ()
            let value = (roll - 1) * (pown 6 power)
            if log then printfn $"rollAcc / acc: {acc} / power: {power} / roll: {roll} / value: {value}"
            rollAcc (acc + value) (power - 1)

    let rec rollMax power =
        let result = rollAcc 0 power
        if log then printfn $"rollMax / power: {power} / result: {result}"
        if result < max
        then result + 1
        else rollMax power

    rollMax ((numDice 1) - 1)

In [ ]:
//// ignore

[1..100]
|> List.map (fun _ -> inverseTransformSamplingRoll2 false 10)
|> List.groupBy id
|> List.map (fun (k, v) -> k, v.Length)
|> List.sortBy fst

index value 0 (1, 8) Item1 1 Item2 8 1 (2, 12) Item1 2 Item2 12 2 (3, 10) Item1 3 Item2 10 3 (4, 9) Item1 4 Item2 9 4 (5, 11) Item1 5 Item2 11 5 (6, 11) Item1 6 Item2 11 6 (7, 9) Item1 7 Item2 9 7 (8, 9) Item1 8 Item2 9 8 (9, 13) Item1 9 Item2 13 9 (10, 8) Item1 10 Item2 8

In [ ]:
//// ignore

inverseTransformSamplingRoll2 true 2000

774

numDice / n: 1 / max: 2000 / pown: 6
numDice / n: 2 / max: 2000 / pown: 36
numDice / n: 3 / max: 2000 / pown: 216
numDice / n: 4 / max: 2000 / pown: 1296
numDice / n: 5 / max: 2000 / pown: 7776
rollAcc / acc: 0 / power: 4 / roll: 3 / value: 2592
rollAcc / acc: 2592 / power: 3 / roll: 2 / value: 216
rollAcc / acc: 2808 / power: 2 / roll: 5 / value: 144
rollAcc / acc: 2952 / power: 1 / roll: 4 / value: 18
rollAcc / acc: 2970 / power: 0 / roll: 3 / value: 2
rollAcc / acc: 2972 / power: -1
rollMax / power: 4 / result: 2972
rollAcc / acc: 0 / power: 4 / roll: 2 / value: 1296
rollAcc / acc: 1296 / power: 3 / roll: 4 / value: 648
rollAcc / acc: 1944 / power: 2 / roll: 5 / value: 144
rollAcc / acc: 2088 / power: 1 / roll: 1 / value: 0
rollAcc / acc: 2088 / power: 0 / roll: 1 / value: 0
rollAcc / acc: 2088 / power: -1
rollMax / power: 4 / result: 2088
rollAcc / acc: 0 / power: 4 / roll: 5 / value: 5184
rollAcc / acc: 5184 / power: 3 / roll: 3 / value: 432
rollAcc / acc: 5616 / power: 2 / roll: 

In [ ]:
//// ignore

let max = 10
let n = 30
let even = (n / max) |> int

let rec inverseTransformSamplingRoll2N current =
    let roll = rollMax inverseTransformSamplingRoll2 max n
    if roll |> List.forall ((=) even)
    then current
    else inverseTransformSamplingRoll2N (current + 1)

inverseTransformSamplingRoll2N 0

84140

In [ ]:
//// ignore

[1..100] |> List.map (fun _ -> inverseTransformSamplingRoll2N 0) |> List.map float |> List.average

225761.82

### Rejection Sampling Roll

In [ ]:
let rejectionSamplingRoll log max =
    let rec numDice n =
        if log then printfn $"numDice / n: {n} / max: {max} / pown: {pown 6 n}"
        if pown 6 n > max
        then n
        else numDice (n + 1)

    let rec rollAcc acc power =
        if power < 0
        then
            if log then printfn $"rollAcc / acc: {acc} / power: {power}"
            acc
        else
            let roll = rollD6 () - 1
            let value = roll * (pown 6 power)
            if log then printfn $"rollAcc / acc: {acc} / power: {power} / roll: {roll} / value: {value}"
            rollAcc (acc + value) (power - 1)

    let rec rollMax power =
        let result = rollAcc 0 power
        if log then printfn $"rollMax / power: {power} / result: {result}"
        if result >= 1 && result <= max
        then result
        else rollMax power

    rollMax ((numDice 1) - 1)

In [ ]:
//// ignore

rejectionSamplingRoll true 2000

1615

numDice / n: 1 / max: 2000 / pown: 6
numDice / n: 2 / max: 2000 / pown: 36
numDice / n: 3 / max: 2000 / pown: 216
numDice / n: 4 / max: 2000 / pown: 1296
numDice / n: 5 / max: 2000 / pown: 7776
rollAcc / acc: 0 / power: 4 / roll: 2 / value: 2592
rollAcc / acc: 2592 / power: 3 / roll: 0 / value: 0
rollAcc / acc: 2592 / power: 2 / roll: 0 / value: 0
rollAcc / acc: 2592 / power: 1 / roll: 5 / value: 30
rollAcc / acc: 2622 / power: 0 / roll: 2 / value: 2
rollAcc / acc: 2624 / power: -1
rollMax / power: 4 / result: 2624
rollAcc / acc: 0 / power: 4 / roll: 2 / value: 2592
rollAcc / acc: 2592 / power: 3 / roll: 5 / value: 1080
rollAcc / acc: 3672 / power: 2 / roll: 2 / value: 72
rollAcc / acc: 3744 / power: 1 / roll: 2 / value: 12
rollAcc / acc: 3756 / power: 0 / roll: 4 / value: 4
rollAcc / acc: 3760 / power: -1
rollMax / power: 4 / result: 3760
rollAcc / acc: 0 / power: 4 / roll: 1 / value: 1296
rollAcc / acc: 1296 / power: 3 / roll: 1 / value: 216
rollAcc / acc: 1512 / power: 2 / roll: 2 /

In [ ]:
//// ignore

[1..100]
|> List.map (fun _ -> rejectionSamplingRoll false 10)
|> List.groupBy id
|> List.map (fun (k, v) -> k, v.Length)
|> List.sortBy fst

index value 0 (1, 8) Item1 1 Item2 8 1 (2, 10) Item1 2 Item2 10 2 (3, 13) Item1 3 Item2 13 3 (4, 14) Item1 4 Item2 14 4 (5, 7) Item1 5 Item2 7 5 (6, 6) Item1 6 Item2 6 6 (7, 16) Item1 7 Item2 16 7 (8, 8) Item1 8 Item2 8 8 (9, 11) Item1 9 Item2 11 9 (10, 7) Item1 10 Item2 7

In [ ]:
//// ignore

let max = 10
let n = 30
let even = (n / max) |> int

let rec rejectionSamplingRollN current =
    let roll = rollMax rejectionSamplingRoll max n
    if roll |> List.forall ((=) even)
    then current
    else rejectionSamplingRollN (current + 1)

rejectionSamplingRollN 0

395797

In [ ]:
//// ignore

[1..100] |> List.map (fun _ -> rejectionSamplingRollN 0) |> List.map float |> List.average

219867.02

In [ ]:
let rejectionSamplingRoll2 log max =
    let rec numDice n =
        if log then printfn $"numDice / n: {n} / max: {max} / pown: {pown 6 n}"
        if pown 6 n > max
        then n
        else numDice (n + 1)

    let rec rollAcc acc power =
        if power < 0
        then
            if log then printfn $"rollAcc / acc: {acc} / power: {power}"
            acc
        else
            let roll = rollD6 ()
            let value = (roll - 1) * (pown 6 power)
            if log then printfn $"rollAcc / acc: {acc} / power: {power} / roll: {roll} / value: {value}"
            rollAcc (acc + value) (power - 1)

    let rec rollMax power =
        let result = rollAcc 0 power
        if log then printfn $"rollMax / power: {power} / result: {result}"
        if result >= 1 && result <= max
        then result
        else rollMax power

    rollMax ((numDice 1) - 1)

In [ ]:
//// ignore

[1..100]
|> List.map (fun _ -> rejectionSamplingRoll2 false 10)
|> List.groupBy id
|> List.map (fun (k, v) -> k, v.Length)
|> List.sortBy fst

index value 0 (1, 6) Item1 1 Item2 6 1 (2, 7) Item1 2 Item2 7 2 (3, 9) Item1 3 Item2 9 3 (4, 9) Item1 4 Item2 9 4 (5, 12) Item1 5 Item2 12 5 (6, 16) Item1 6 Item2 16 6 (7, 14) Item1 7 Item2 14 7 (8, 9) Item1 8 Item2 9 8 (9, 12) Item1 9 Item2 12 9 (10, 6) Item1 10 Item2 6

In [ ]:
//// ignore

rejectionSamplingRoll2 true 2000

1686

numDice / n: 1 / max: 2000 / pown: 6
numDice / n: 2 / max: 2000 / pown: 36
numDice / n: 3 / max: 2000 / pown: 216
numDice / n: 4 / max: 2000 / pown: 1296
numDice / n: 5 / max: 2000 / pown: 7776
rollAcc / acc: 0 / power: 4 / roll: 4 / value: 3888
rollAcc / acc: 3888 / power: 3 / roll: 6 / value: 1080
rollAcc / acc: 4968 / power: 2 / roll: 4 / value: 108
rollAcc / acc: 5076 / power: 1 / roll: 6 / value: 30
rollAcc / acc: 5106 / power: 0 / roll: 3 / value: 2
rollAcc / acc: 5108 / power: -1
rollMax / power: 4 / result: 5108
rollAcc / acc: 0 / power: 4 / roll: 2 / value: 1296
rollAcc / acc: 1296 / power: 3 / roll: 2 / value: 216
rollAcc / acc: 1512 / power: 2 / roll: 5 / value: 144
rollAcc / acc: 1656 / power: 1 / roll: 6 / value: 30
rollAcc / acc: 1686 / power: 0 / roll: 1 / value: 0
rollAcc / acc: 1686 / power: -1
rollMax / power: 4 / result: 1686


In [ ]:
//// ignore

[1..100]
|> List.map (fun _ -> rejectionSamplingRoll2 false 10)
|> List.groupBy id
|> List.map (fun (k, v) -> k, v.Length)
|> List.sortBy fst

index value 0 (1, 10) Item1 1 Item2 10 1 (2, 8) Item1 2 Item2 8 2 (3, 15) Item1 3 Item2 15 3 (4, 11) Item1 4 Item2 11 4 (5, 8) Item1 5 Item2 8 5 (6, 12) Item1 6 Item2 12 6 (7, 8) Item1 7 Item2 8 7 (8, 7) Item1 8 Item2 7 8 (9, 10) Item1 9 Item2 10 9 (10, 11) Item1 10 Item2 11

In [ ]:
//// ignore

let max = 10
let n = 30
let even = (n / max) |> int

let rec rejectionSamplingRoll2N current =
    let roll = rollMax rejectionSamplingRoll2 max n
    if roll |> List.forall ((=) even)
    then current
    else rejectionSamplingRoll2N (current + 1)

rejectionSamplingRoll2N 0

203584

In [ ]:
//// ignore

[1..100] |> List.map (fun _ -> rejectionSamplingRoll2N 0) |> List.map float |> List.average

208646.59